# GKE - flask - Let's Encrypt HTTPS
#### Deploying a flask app on a K8s Cluster in the Google Cloud and access it through HTTPS

In [1]:
!pwd

/home/wgiersche/workspace/tutorials/microblog


In [ ]:
!docker build 

```
gcloud container clusters create microblog-cluster --num-nodes=2
```

### Links: 

#### 1) [The flask mega-tutorial](https://blog.miguelgrinberg.com/)
#### 2) [HTTPS on GKE](https://medium.com/a-little-bit/https-and-gke-6a0dc702603f)
#### 3) [Install gcloud sdk](https://cloud.google.com/sdk/docs/quickstart-linux)
#### 4) [Cluster access with kubectl](https://cloud.google.com/kubernetes-engine/docs/how-to/cluster-access-for-kubectl)
#### 5) [Install docker on Debian](https://docs.docker.com/install/linux/docker-ce/debian/#install-docker-ce-1)
#### 6) [Viacheslav's Ready-to-go Approach for machine learning on GCE](https://blog.kovalevskyi.com/semi-managed-jupyter-lab-with-access-to-google-cloud-resources-cc6f9e439416)
#### 7) [Pulling and Pushing Images](https://cloud.google.com/container-registry/docs/pushing-and-pulling)
#### 8) [Sandeep's Best Practices for K8s](https://medium.com/google-cloud/kubernetes-best-practices-season-one-11119aee1d10)
#### 9) []()

Follow [the gcloud sdk installation guide](https://cloud.google.com/sdk/docs/quickstart-linux) to install the gcloud SDK. Clean up older executables when prompted to do so.

Perform the following to install the gcloud, kubectl and docker 
```
# Cloud SDK (gcloud and friends)
export CLOUD_SDK_REPO="cloud-sdk-$(lsb_release -c -s)"
echo "deb http://packages.cloud.google.com/apt $CLOUD_SDK_REPO main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
sudo apt-get update && sudo apt-get install -y google-cloud-sdk
sudo apt-get install -y kubectl

# docker
sudo apt-get install apt-transport-https ca-certificates curl gnupg2 software-properties-common
curl -fsSL https://download.docker.com/linux/debian/gpg | sudo apt-key add -
sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/debian $(lsb_release -cs) stable"
sudo apt-get update
sudo apt-get install -y docker-ce

# Not sure whether this is really needed. Change to your service account.
# Use gcloud auth list to identify your service account
SERVICE_ACCOUNT=serviceAccount:1094881674505-compute@developer.gserviceaccount.com
gsutil iam ch $SERVICE_ACCOUNT:admin gs://going-tfx

# docker credential helper provides access to the image repo
VERSION=1.5.0
OS=linux
ARCH=amd64
curl -fsSL "https://github.com/GoogleCloudPlatform/docker-credential-gcr/releases/download/v${VERSION}/docker-credential-gcr_${OS}_${ARCH}-${VERSION}.tar.gz" | tar xz
sudo mv docker-credential-gcr /usr/bin/docker-credential-gcr
sudo chmod +x /usr/bin/docker-credential-gcr
docker-credential-gcr
```
The last command should indicate that your active account is a service account, created for you on the fly to access the GCP resources.

Now the following command should succeed and create a small 2-node cluster for you

In [2]:
%%bash
which kubectl
which gcloud

/usr/bin/kubectl
/usr/bin/gcloud


In [4]:
!gcloud container clusters create microblog-cluster --num-nodes=2

This will enable the autorepair feature for nodes. Please see https://cloud.google.com/kubernetes-engine/docs/node-auto-repair for more information on node autorepairs.
Creating cluster microblog-cluster in us-central1-a...done.
Created [https://container.googleapis.com/v1/projects/going-tfx/zones/us-central1-a/clusters/microblog-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-central1-a/microblog-cluster?project=going-tfx
kubeconfig entry generated for microblog-cluster.
NAME               LOCATION       MASTER_VERSION  MASTER_IP      MACHINE_TYPE   NODE_VERSION  NUM_NODES  STATUS
microblog-cluster  us-central1-a  1.10.9-gke.5    35.239.25.105  n1-standard-1  1.10.9-gke.5  2          RUNNING


In [27]:
!gcloud container clusters get-credentials microblog-cluster --zone us-central1-a --project going-tfx

Fetching cluster endpoint and auth data.
kubeconfig entry generated for microblog-cluster.


In [29]:
!kubectl apply -f https://raw.githubusercontent.com/kubernetes/dashboard/master/src/deploy/recommended/kubernetes-dashboard.yaml

secret/kubernetes-dashboard-certs created
serviceaccount/kubernetes-dashboard created
deployment.apps/kubernetes-dashboard created
service/kubernetes-dashboard created
Error from server (Forbidden): error when creating "https://raw.githubusercontent.com/kubernetes/dashboard/master/src/deploy/recommended/kubernetes-dashboard.yaml": roles.rbac.authorization.k8s.io is forbidden: User "113438764705426694571" cannot create roles.rbac.authorization.k8s.io in the namespace "kube-system": Required "container.roles.create" permission.
Error from server (Forbidden): error when creating "https://raw.githubusercontent.com/kubernetes/dashboard/master/src/deploy/recommended/kubernetes-dashboard.yaml": rolebindings.rbac.authorization.k8s.io is forbidden: User "113438764705426694571" cannot create rolebindings.rbac.authorization.k8s.io in the namespace "kube-system": Required "container.roleBindings.create" permission.


In [ ]:
kubectl run microblog --image=us.gcr.io/going-tfx/microblog:v1 --port=5000

In [31]:
!curl https://raw.githubusercontent.com/helm/helm/master/scripts/get | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7236  100  7236    0     0   113k      0 --:--:-- --:--:-- --:--:--  112k
Helm v2.12.0 is already latest
Run 'helm init' to configure helm.


In [32]:
!helm init

$HELM_HOME has been configured at /home/wgiersche/.helm.
(Use --client-only to suppress this message, or --upgrade to upgrade Tiller to the current version.)
Happy Helming!


In [33]:
!which helm

/usr/local/bin/helm


In [37]:
!kubectl -n kube-system delete deployment tiller-deploy
!kubectl delete clusterrolebinding tiller
!kubectl -n kube-system delete serviceaccount tiller

Error from server (NotFound): deployments.extensions "tiller-deploy" not found
clusterrolebinding.rbac.authorization.k8s.io "tiller" deleted
serviceaccount "tiller" deleted


In [38]:
!kubectl -n kube-system create serviceaccount tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account=tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/wgiersche/.helm.
(Use --client-only to suppress this message, or --upgrade to upgrade Tiller to the current version.)
Happy Helming!


In [39]:
!helm ls

In [42]:
!helm install --name cert-manager --namespace cert-manager stable/cert-manager

Error: release cert-manager failed: customresourcedefinitions.apiextensions.k8s.io "certificates.certmanager.k8s.io" already exists


In [41]:
!helm del --purge cert-manager;

release "cert-manager" deleted
